# RO System Simulation Template - Simplified Version

This simplified notebook template uses fixed pressure drops per stage to avoid
the need to specify length and width of the membrane (simply provide the total area per stage).

In [ ]:
# Parameters cell - will be replaced by papermill
project_root = "/path/to/project"  # Will be replaced by papermill
configuration = {}
feed_salinity_ppm = 5000
feed_temperature_c = 25.0
membrane_type = "brackish"
membrane_properties = None
optimize_pumps = False

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from pyomo.environ import *
from pyomo.network import Arc
import sys
import os

# Add parent directory to path for utils imports using project_root parameter
sys.path.insert(0, project_root)

# WaterTAP imports
from watertap.core.solvers import get_solver
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType
)
from watertap.unit_models.pressure_changer import Pump
from watertap.property_models.multicomp_aq_sol_prop_pack import MCASParameterBlock
import watertap.property_models.seawater_prop_pack as props_sw
from watertap.core import ModuleType

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.core.util import DiagnosticsToolbox
from idaes.models.unit_models import Feed, Product

# Import membrane properties handler
from utils.membrane_properties_handler import get_membrane_properties

import warnings
warnings.filterwarnings('ignore')

# Results storage
results = {}

## Build Simplified WaterTAP Model

In [ ]:
def build_ro_model_simple(config_data, feed_salinity_ppm, feed_temperature_c, membrane_type, membrane_properties=None):
    """
    Build simplified WaterTAP RO model using fixed pressure drops.
    This avoids the need to specify membrane length and width.
    """
    # Create concrete model
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    
    # Property package - using seawater
    m.fs.properties = props_sw.SeawaterParameterBlock()
    
    # Feed conditions
    feed_flow_m3_s = config_data['feed_flow_m3h'] / 3600  # Convert to m³/s
    feed_mass_frac = feed_salinity_ppm / 1e6  # Convert ppm to mass fraction
    
    # Create feed unit
    m.fs.feed = Feed(property_package=m.fs.properties)
    
    # Build stages
    n_stages = config_data['stage_count']
    
    # Get membrane properties using new system
    A_w, B_s = get_membrane_properties(membrane_type, membrane_properties)
    print(f"Using membrane properties for '{membrane_type}': A_w={A_w:.2e} m/s/Pa, B_s={B_s:.2e} m/s")
    
    # First, create all RO stages and pumps using setattr for proper parent assignment
    for i in range(1, n_stages + 1):
        # Create feed pump for stage using setattr
        setattr(m.fs, f"pump{i}", Pump(property_package=m.fs.properties))
        
        # Create RO stage with simplified configuration:
        # - fixed_per_stage pressure drop
        # - none or fixed concentration polarization
        # - none mass transfer coefficient
        # - Add spiral wound module type
        setattr(m.fs, f"ro_stage{i}", ReverseOsmosis0D(
            property_package=m.fs.properties,
            has_pressure_change=True,
            concentration_polarization_type=ConcentrationPolarizationType.none,
            mass_transfer_coefficient=MassTransferCoefficient.none,
            pressure_change_type=PressureChangeType.fixed_per_stage,
            module_type=ModuleType.spiral_wound  # Add spiral wound module type
        ))
        
        # Create permeate product for each stage
        setattr(m.fs, f"stage_product{i}", Product(property_package=m.fs.properties))
    
    # Create final concentrate product
    m.fs.concentrate_product = Product(property_package=m.fs.properties)
    
    # Connect feed to first pump
    m.fs.feed_to_pump1 = Arc(
        source=m.fs.feed.outlet,
        destination=m.fs.pump1.inlet
    )
    
    # Connect first pump to first RO
    m.fs.pump1_to_ro1 = Arc(
        source=m.fs.pump1.outlet,
        destination=m.fs.ro_stage1.inlet
    )
    
    # Connect permeate from first RO to product
    m.fs.ro1_perm_to_prod = Arc(
        source=m.fs.ro_stage1.permeate,
        destination=m.fs.stage_product1.inlet
    )
    
    # Connect stages
    if n_stages > 1:
        for i in range(1, n_stages):
            # Connect concentrate of stage i to pump i+1
            setattr(
                m.fs, f"stage{i}_to_pump{i+1}",
                Arc(
                    source=getattr(m.fs, f"ro_stage{i}").retentate,
                    destination=getattr(m.fs, f"pump{i+1}").inlet
                )
            )
            # Connect pump i+1 to stage i+1
            setattr(
                m.fs, f"pump{i+1}_to_stage{i+1}",
                Arc(
                    source=getattr(m.fs, f"pump{i+1}").outlet,
                    destination=getattr(m.fs, f"ro_stage{i+1}").inlet
                )
            )
            # Connect permeate to product
            setattr(
                m.fs, f"ro{i+1}_perm_to_prod{i+1}",
                Arc(
                    source=getattr(m.fs, f"ro_stage{i+1}").permeate,
                    destination=getattr(m.fs, f"stage_product{i+1}").inlet
                )
            )
    
    # Connect final concentrate to product
    final_stage = n_stages
    m.fs.final_conc_arc = Arc(
        source=getattr(m.fs, f"ro_stage{final_stage}").retentate,
        destination=m.fs.concentrate_product.inlet
    )
    
    # Apply arcs to expand the network
    TransformationFactory("network.expand_arcs").apply_to(m)
    
    # NOW set membrane properties after model structure is built
    for i in range(1, n_stages + 1):
        stage_data = config_data['stages'][i-1]
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Use membrane properties from handler
        ro.A_comp.fix(A_w)  # m/s/Pa
        ro.B_comp[0, 'TDS'].fix(B_s)  # m/s
        
        # For fixed_per_stage pressure change type with spiral wound modules:
        # While we still use the simplified approach with fixed pressure drop,
        # the spiral wound module type ensures proper mass transfer correlations
        # are used for the membrane calculations
        
        # Set membrane area directly
        ro.area.fix(stage_data['membrane_area_m2'])
        
        # Set pressure drop for the stage
        # Use a reasonable default based on stage number and recovery
        if i == 1:
            ro.deltaP.fix(-0.5e5)  # -0.5 bar pressure drop
        elif i == 2:
            ro.deltaP.fix(-0.7e5)  # -0.7 bar pressure drop
        else:
            ro.deltaP.fix(-1.0e5)  # -1.0 bar pressure drop
        
        # Fix permeate pressure (typically atmospheric)
        ro.permeate.pressure.fix(101325)  # 1 atm
    
    # Set feed conditions
    m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(
        feed_flow_m3_s * 1000 * (1 - feed_mass_frac)
    )
    m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', 'TDS'].fix(
        feed_flow_m3_s * 1000 * feed_mass_frac
    )
    m.fs.feed.outlet.temperature.fix(273.15 + feed_temperature_c)
    m.fs.feed.outlet.pressure.fix(101325)  # 1 atm
    
    # Set pump efficiencies
    for i in range(1, n_stages + 1):
        getattr(m.fs, f"pump{i}").efficiency_pump.fix(0.8)
    
    # Set scaling
    m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
    m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
    calculate_scaling_factors(m)
    
    return m

## Initialize and Solve Model

In [ ]:
def initialize_and_solve_simple(m, config_data, optimize_pumps=False):
    """
    Initialize and solve the simplified RO model with intelligent pressure guesses
    based on osmotic pressure calculations.
    """
    solver = get_solver()
    
    # Check initial degrees of freedom  
    print(f"Initial degrees of freedom: {degrees_of_freedom(m)}")
    
    # We should have DOF = number of pumps (their outlet pressures are not fixed)
    expected_dof = config_data['stage_count']
    if degrees_of_freedom(m) != expected_dof:
        print(f"Warning: Expected {expected_dof} degrees of freedom, got {degrees_of_freedom(m)}")
    
    # Initialize feed
    m.fs.feed.initialize()
    
    # Set scaling factors for better convergence
    m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1, index=('Liq', 'H2O'))
    m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e1, index=('Liq', 'TDS'))
    m.fs.properties.set_default_scaling('pressure', 1e-5)
    m.fs.properties.set_default_scaling('temperature', 1e-2)
    
    # Stage-specific pressure guesses based on osmotic pressure
    n_stages = config_data['stage_count']
    
    # Propagate state from feed to first pump
    propagate_state(arc=m.fs.feed_to_pump1)
    
    # Initialize pumps and RO stages sequentially
    for i in range(1, n_stages + 1):
        pump = getattr(m.fs, f"pump{i}")
        ro = getattr(m.fs, f"ro_stage{i}")
        stage_data = config_data['stages'][i-1]
        
        # Get inlet conditions
        inlet_pressure = value(pump.inlet.pressure[0])
        inlet_flow_h2o = value(pump.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        inlet_flow_tds = value(pump.inlet.flow_mass_phase_comp[0, 'Liq', 'TDS'])
        inlet_temp = value(pump.inlet.temperature[0])
        
        # Calculate feed TDS concentration
        feed_tds_ppm = inlet_flow_tds / (inlet_flow_h2o + inlet_flow_tds) * 1e6
        
        # Get recovery from configuration
        stage_recovery = stage_data.get('stage_recovery', 0.5)
        
        # Calculate concentrate TDS concentration based on recovery
        # Mass balance: feed_tds = permeate_tds + concentrate_tds
        # Assuming high rejection (>99%), most TDS goes to concentrate
        # concentrate_flow = feed_flow * (1 - recovery)
        concentrate_tds_ppm = feed_tds_ppm / (1 - stage_recovery)
        
        # Calculate average TDS in the membrane (geometric mean)
        avg_tds_ppm = (feed_tds_ppm * concentrate_tds_ppm) ** 0.5
        
        # Calculate osmotic pressure using simplified correlation
        # π (bar) ≈ 0.7 * TDS (g/L) for NaCl-type solutions
        # Convert ppm to g/L (approximately same for dilute solutions)
        osmotic_pressure_bar = 0.7 * avg_tds_ppm / 1000
        
        # Calculate required net driving pressure
        # Need positive driving pressure for water flux
        # Typical flux requires 10-20 bar net driving pressure
        min_net_driving_pressure = 15  # bar
        
        # Required feed pressure = permeate pressure + osmotic pressure + net driving pressure + pressure drop
        permeate_pressure_bar = 1.0  # atmospheric
        pressure_drop_bar = abs(value(ro.deltaP[0])) / 1e5
        
        required_pressure_bar = (permeate_pressure_bar + 
                                osmotic_pressure_bar + 
                                min_net_driving_pressure + 
                                pressure_drop_bar)
        
        # Add safety factor for higher stages
        safety_factor = 1.0 + 0.1 * (i - 1)  # 10% increase per stage
        pressure_guess = required_pressure_bar * safety_factor * 1e5  # Convert to Pa
        
        # Ensure minimum pressure boost
        min_pressure = inlet_pressure + 5e5  # At least 5 bar boost
        pressure_guess = max(pressure_guess, min_pressure)
        pressure_guess = min(pressure_guess, 80e5)  # Max 80 bar
        
        # Set bounds and initial value for pump outlet pressure
        pump.outlet.pressure[0].setlb(min_pressure)
        pump.outlet.pressure[0].setub(80e5)
        pump.outlet.pressure[0].set_value(pressure_guess)
        
        # Initialize pump with explicit outlet state
        pump.initialize(
            state_args={
                "pressure": pressure_guess,
                "temperature": inlet_temp,
                "flow_mass_phase_comp": {
                    ('Liq', 'H2O'): inlet_flow_h2o,
                    ('Liq', 'TDS'): inlet_flow_tds
                }
            }
        )
        
        print(f"\n  Stage {i} initialization:")
        print(f"    Feed TDS: {feed_tds_ppm:.0f} ppm")
        print(f"    Expected concentrate TDS: {concentrate_tds_ppm:.0f} ppm")
        print(f"    Estimated osmotic pressure: {osmotic_pressure_bar:.1f} bar")
        print(f"    Initial pump pressure: {pressure_guess/1e5:.1f} bar")
        
        # Propagate state from pump to RO
        if i == 1:
            propagate_state(arc=m.fs.pump1_to_ro1)
        else:
            arc_name = f"pump{i}_to_stage{i}"
            if hasattr(m.fs, arc_name):
                propagate_state(arc=getattr(m.fs, arc_name))
        
        # Apply scaling to RO unit
        calculate_scaling_factors(ro)
        
        # IMPORTANT: Relax flux bounds for multi-stage systems
        # This prevents FBBT infeasibility issues
        for t in ro.flowsheet().time:
            for x in ro.length_domain:
                ro.flux_mass_phase_comp[t, x, 'Liq', 'H2O'].setlb(1e-8)
                ro.flux_mass_phase_comp[t, x, 'Liq', 'H2O'].setub(0.1)
                ro.flux_mass_phase_comp[t, x, 'Liq', 'TDS'].setlb(1e-12)
                ro.flux_mass_phase_comp[t, x, 'Liq', 'TDS'].setub(0.01)
        
        # Initialize RO with robust settings
        try:
            ro.initialize(
                optarg={
                    'tol': 1e-6,
                    'constr_viol_tol': 1e-6,
                    'nlp_scaling_method': 'user-scaling',
                    'linear_solver': 'ma27',
                    'max_iter': 200
                }
            )
            print(f"    RO initialization successful")
        except Exception as e:
            print(f"    Warning: RO initialization issue: {str(e)}")
            # Continue anyway - solver might still find a solution
        
        # Initialize stage product
        if i == 1:
            propagate_state(arc=m.fs.ro1_perm_to_prod)
        else:
            arc_name = f"ro{i}_perm_to_prod{i}"
            if hasattr(m.fs, arc_name):
                propagate_state(arc=getattr(m.fs, arc_name))
        
        getattr(m.fs, f"stage_product{i}").initialize()
        
        # If not the last stage, propagate to next pump
        if i < n_stages:
            arc_name = f"stage{i}_to_pump{i+1}"
            if hasattr(m.fs, arc_name):
                propagate_state(arc=getattr(m.fs, arc_name))
    
    # Initialize final concentrate product
    propagate_state(arc=m.fs.final_conc_arc)
    m.fs.concentrate_product.initialize()
    
    # Add recovery constraints from configuration
    for i in range(1, n_stages + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        stage_data = config_data['stages'][i-1]
        
        # Use recovery from configuration (tool 1 output)
        target_recovery = stage_data.get('stage_recovery', 0.5)
        
        # Add recovery constraint
        setattr(
            m.fs, f"recovery_constraint_stage{i}",
            Constraint(
                expr=ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'] == target_recovery
            )
        )
        
        print(f"\n  Stage {i} recovery constraint: {target_recovery:.1%}")
    
    # Check DOF after adding constraints
    print(f"\nDegrees of freedom after recovery constraints: {degrees_of_freedom(m)}")
    
    # Solve with robust solver settings
    print("\nSolving model to determine required pump pressures...")
    results = solver.solve(m, tee=False, options={
        'tol': 1e-6,
        'constr_viol_tol': 1e-6,
        'max_iter': 300,
        'linear_solver': 'ma27'
    })
    
    # Report results
    if results.solver.termination_condition == TerminationCondition.optimal:
        print("\nSolution found!")
        print("\nCalculated Pump Pressures:")
        for i in range(1, n_stages + 1):
            pump = getattr(m.fs, f"pump{i}")
            ro = getattr(m.fs, f"ro_stage{i}")
            
            feed_pressure = value(pump.outlet.pressure[0])
            inlet_pressure = value(pump.inlet.pressure[0])
            power_kw = value(pump.work_mechanical[0]) / 1000
            recovery = value(ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'])
            
            # Get flux for diagnostics
            flux = value(ro.flux_mass_phase_comp[0, 0, 'Liq', 'H2O'])
            
            # Calculate actual osmotic pressure at membrane interface
            feed_side_outlet = ro.feed_side.properties_out[0]
            interface = ro.feed_side.properties_interface[0]
            actual_osmotic_p = value(interface.pressure_osm_phase['Liq']) / 1e5
            
            print(f"\n  Stage {i}:")
            print(f"    Pump inlet pressure: {inlet_pressure/1e5:.1f} bar")
            print(f"    Required feed pressure: {feed_pressure/1e5:.1f} bar")
            print(f"    Pressure boost: {(feed_pressure-inlet_pressure)/1e5:.1f} bar")
            print(f"    Pump power: {power_kw:.1f} kW")
            print(f"    Water recovery: {recovery:.1%}")
            print(f"    Water flux: {flux:.5f} kg/m²/s")
            print(f"    Actual osmotic pressure: {actual_osmotic_p:.1f} bar")
    else:
        print(f"\nSolver failed: {results.solver.termination_condition}")
        # Try to provide diagnostic information
        print("\nDiagnostic information:")
        for i in range(1, n_stages + 1):
            pump = getattr(m.fs, f"pump{i}")
            print(f"  Stage {i} pump pressure: {value(pump.outlet.pressure[0])/1e5:.1f} bar")
    
    return results

In [ ]:
def extract_results(m, config_data):
    """Extract simulation results from solved model."""
    results = {
        "status": "success",
        "property_package": "seawater",
        "model_type": "simplified_fixed_pressure_drop",
        "configuration": config_data,
        "performance": {},
        "economics": {},
        "stage_results": [],
        "mass_balance": {},
        "pump_results": []
    }
    
    # Calculate overall performance
    total_feed = value(m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', 'H2O']) + \
                 value(m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', 'TDS'])
    
    total_permeate_water = 0
    total_permeate_tds = 0
    total_pump_power = 0
    
    # Extract stage results
    for i in range(1, config_data['stage_count'] + 1):
        pump = getattr(m.fs, f"pump{i}")
        ro = getattr(m.fs, f"ro_stage{i}")
        product = getattr(m.fs, f"stage_product{i}")
        
        # Get stage flows
        perm_water = value(product.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        perm_tds = value(product.inlet.flow_mass_phase_comp[0, 'Liq', 'TDS'])
        
        # Calculate concentrations
        perm_conc = perm_tds / (perm_water + perm_tds) * 1e6  # ppm
        
        # Get pressures
        feed_pressure = value(pump.outlet.pressure[0])
        perm_pressure = value(ro.permeate.pressure[0])
        pressure_drop = value(ro.deltaP[0])
        
        # Stage results
        stage_result = {
            "stage_number": i,
            "feed_pressure_bar": feed_pressure / 1e5,
            "permeate_pressure_bar": perm_pressure / 1e5,
            "pressure_drop_bar": -pressure_drop / 1e5,  # Make positive
            "recovery": value(ro.recovery_mass_phase_comp[0, 'Liq', 'H2O']),
            "membrane_area_m2": value(ro.area),
            "permeate_flow_m3h": perm_water / 1000 * 3600,  # Convert kg/s to m³/h
            "permeate_tds_ppm": perm_conc,
            "pump_power_kw": value(pump.work_mechanical[0]) / 1000
        }
        
        results["stage_results"].append(stage_result)
        
        # Pump results
        pump_result = {
            "pump_number": i,
            "inlet_pressure_bar": value(pump.inlet.pressure[0]) / 1e5,
            "outlet_pressure_bar": feed_pressure / 1e5,
            "pressure_boost_bar": (feed_pressure - value(pump.inlet.pressure[0])) / 1e5,
            "power_kw": value(pump.work_mechanical[0]) / 1000,
            "efficiency": value(pump.efficiency_pump[0])
        }
        results["pump_results"].append(pump_result)
        
        # Accumulate totals
        total_permeate_water += perm_water
        total_permeate_tds += perm_tds
        total_pump_power += value(pump.work_mechanical[0])
    
    # Overall performance
    results["performance"]["total_recovery"] = total_permeate_water / value(
        m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', 'H2O']
    )
    results["performance"]["permeate_flow_m3h"] = total_permeate_water / 1000 * 3600
    results["performance"]["permeate_tds_ppm"] = total_permeate_tds / total_permeate_water * 1e6
    results["performance"]["total_pump_power_kw"] = total_pump_power / 1000
    
    # Economics
    results["economics"]["specific_energy_kwh_m3"] = (total_pump_power / 1000) / (
        total_permeate_water / 1000 * 3600
    )
    
    # Mass balance
    concentrate = m.fs.concentrate_product
    conc_water = value(concentrate.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'])
    conc_tds = value(concentrate.inlet.flow_mass_phase_comp[0, 'Liq', 'TDS'])
    
    results["mass_balance"]["feed_flow_kgs"] = total_feed
    results["mass_balance"]["permeate_flow_kgs"] = total_permeate_water + total_permeate_tds
    results["mass_balance"]["concentrate_flow_kgs"] = conc_water + conc_tds
    results["mass_balance"]["balance_error"] = abs(
        total_feed - (total_permeate_water + total_permeate_tds + conc_water + conc_tds)
    ) / total_feed
    
    return results

In [ ]:
def initialize_and_solve_simple(m, config_data, optimize_pumps=False):
    """
    Initialize and solve the simplified RO model.
    The pump outlet pressures are left as decision variables for the solver.
    """
    solver = get_solver()
    
    # Check initial degrees of freedom  
    print(f"Initial degrees of freedom: {degrees_of_freedom(m)}")
    
    # We should have DOF = number of pumps (their outlet pressures are not fixed)
    expected_dof = config_data['stage_count']
    if degrees_of_freedom(m) != expected_dof:
        print(f"Warning: Expected {expected_dof} degrees of freedom, got {degrees_of_freedom(m)}")
    
    # Initialize feed
    m.fs.feed.initialize()
    
    # Set scaling factors for better convergence
    m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1, index=('Liq', 'H2O'))
    m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e1, index=('Liq', 'TDS'))
    m.fs.properties.set_default_scaling('pressure', 1e-5)
    m.fs.properties.set_default_scaling('temperature', 1e-2)
    
    # Set initial guesses for pump outlet pressures based on stage
    # These are just initial guesses, not fixed values
    pressure_guesses = {
        1: 15e5,  # 15 bar for stage 1
        2: 20e5,  # 20 bar for stage 2 
        3: 25e5   # 25 bar for stage 3
    }
    
    # Propagate state from feed to first pump
    propagate_state(arc=m.fs.feed_to_pump1)
    
    # Initialize pumps and RO stages sequentially
    for i in range(1, config_data['stage_count'] + 1):
        pump = getattr(m.fs, f"pump{i}")
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Set bounds and initial value for pump outlet pressure
        pump.outlet.pressure[0].setlb(5e5)   # 5 bar minimum
        pump.outlet.pressure[0].setub(80e5)  # 80 bar maximum
        pump.outlet.pressure[0].set_value(pressure_guesses.get(i, 20e5))
        
        # Get inlet conditions for pump
        inlet_flow_h2o = value(pump.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        inlet_flow_tds = value(pump.inlet.flow_mass_phase_comp[0, 'Liq', 'TDS'])
        inlet_temp = value(pump.inlet.temperature[0])
        
        # Initialize pump with explicit outlet state to avoid pressure bounds issue
        pump.initialize(
            state_args={
                "pressure": pressure_guesses.get(i, 20e5),
                "temperature": inlet_temp,
                "flow_mass_phase_comp": {
                    ('Liq', 'H2O'): inlet_flow_h2o,
                    ('Liq', 'TDS'): inlet_flow_tds
                }
            }
        )
        
        # Propagate state from pump to RO
        if i == 1:
            propagate_state(arc=m.fs.pump1_to_ro1)
        else:
            arc_name = f"pump{i}_to_stage{i}"
            if hasattr(m.fs, arc_name):
                propagate_state(arc=getattr(m.fs, arc_name))
        
        # Apply scaling to RO unit
        calculate_scaling_factors(ro)
        
        # Initialize RO with relaxed tolerances
        ro.initialize(
            optarg={
                'tol': 1e-6,
                'constr_viol_tol': 1e-6,
                'nlp_scaling_method': 'user-scaling',
                'linear_solver': 'ma27'
            }
        )
        
        # Initialize stage product
        if i == 1:
            propagate_state(arc=m.fs.ro1_perm_to_prod)
        else:
            arc_name = f"ro{i}_perm_to_prod{i}"
            if hasattr(m.fs, arc_name):
                propagate_state(arc=getattr(m.fs, arc_name))
        
        getattr(m.fs, f"stage_product{i}").initialize()
        
        # If not the last stage, propagate to next pump
        if i < config_data['stage_count']:
            arc_name = f"stage{i}_to_pump{i+1}"
            if hasattr(m.fs, arc_name):
                propagate_state(arc=getattr(m.fs, arc_name))
    
    # Initialize final concentrate product
    propagate_state(arc=m.fs.final_conc_arc)
    m.fs.concentrate_product.initialize()
    
    # Add constraints to ensure target recovery for each stage
    # Use lower recovery targets to improve feasibility
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        stage_data = config_data['stages'][i-1]
        
        # Use a more conservative recovery target
        target_recovery = stage_data.get('stage_recovery', 0.5)
        # Limit recovery to reasonable values
        target_recovery = min(target_recovery, 0.6 if i == 1 else 0.5)
        
        # Add recovery constraint
        setattr(
            m.fs, f"recovery_constraint_stage{i}",
            Constraint(
                expr=ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'] == target_recovery
            )
        )
    
    # Check DOF after adding constraints
    print(f"Degrees of freedom after recovery constraints: {degrees_of_freedom(m)}")
    
    # Solve
    print("\nSolving model to determine required pump pressures...")
    results = solver.solve(m, tee=False)
    
    # Report results
    if results.solver.termination_condition == TerminationCondition.optimal:
        print("\nSolution found!")
        print("\nCalculated Pump Pressures:")
        for i in range(1, config_data['stage_count'] + 1):
            pump = getattr(m.fs, f"pump{i}")
            ro = getattr(m.fs, f"ro_stage{i}")
            
            feed_pressure = value(pump.outlet.pressure[0])
            inlet_pressure = value(pump.inlet.pressure[0])
            power_kw = value(pump.work_mechanical[0]) / 1000
            recovery = value(ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'])
            
            print(f"\n  Stage {i}:")
            print(f"    Pump inlet pressure: {inlet_pressure/1e5:.1f} bar")
            print(f"    Required feed pressure: {feed_pressure/1e5:.1f} bar")
            print(f"    Pressure boost: {(feed_pressure-inlet_pressure)/1e5:.1f} bar")
            print(f"    Pump power: {power_kw:.1f} kW")
            print(f"    Water recovery: {recovery:.1%}")
    else:
        print(f"\nSolver failed: {results.solver.termination_condition}")
    
    if optimize_pumps and results.solver.termination_condition == TerminationCondition.optimal:
        # Additional optimization can be performed here if needed
        # For example, minimizing power while maintaining recovery
        pass
    
    return results

## Run Simulation

In [ ]:
# Build model
try:
    print("Building simplified RO model...")
    m = build_ro_model_simple(configuration, feed_salinity_ppm, feed_temperature_c, membrane_type, membrane_properties)
    
    print("\nInitializing and solving model...")
    solve_results = initialize_and_solve_simple(m, configuration, optimize_pumps)
    
    if solve_results.solver.termination_condition == TerminationCondition.optimal:
        print("\nExtracting results...")
        results = extract_results(m, configuration)
        results["message"] = "Simulation completed successfully"
        results["model_type"] = "simplified_fixed_pressure_drop"
        results["membrane_type"] = membrane_type
        results["membrane_properties"] = {
            "A_w": value(m.fs.ro_stage1.A_comp[0, 'H2O']),
            "B_s": value(m.fs.ro_stage1.B_comp[0, 'TDS'])
        }
    else:
        results = {
            "status": "error",
            "message": f"Solver failed: {solve_results.solver.termination_condition}",
            "performance": {},
            "economics": {},
            "stage_results": [],
            "mass_balance": {}
        }
        
except Exception as e:
    results = {
        "status": "error",
        "message": f"Simulation error: {str(e)}",
        "performance": {},
        "economics": {},
        "stage_results": [],
        "mass_balance": {}
    }

print("\nSimulation complete.")

## Display Results

In [ ]:
# Display results summary
import json
print("\n" + "="*50)
print("SIMULATION RESULTS - SIMPLIFIED MODEL")
print("="*50)
print(json.dumps(results, indent=2))

In [ ]:
# Results cell - tagged for papermill to extract
results